In [1]:
import pickle
from collections import Counter

import pandas as pd
import datetime

In [2]:
# with open("../data/model.pkl", "rb") as f:
#     model = pickle.load(f)

with open("../data/model_tuning.pkl", "rb") as f:
    model = pickle.load(f)

In [3]:
df = pd.read_pickle("../data/trainset.pkl")

In [4]:
anime_list = df[
    ["Name", "Japanese_name", "anime_id", "Aired", "Score"]
].drop_duplicates()

anime_list.drop(
    anime_list[anime_list["Aired"] == "Unknown"].index, axis=0, inplace=True
)

In [5]:
anime_list[["start_date", "end_date"]] = anime_list["Aired"].str.split(
    "to", expand=True
)
anime_list["start_date"] = anime_list["start_date"].str.rstrip()
anime_list["end_date"] = anime_list["end_date"].str.rstrip()
anime_list["start_yaer"] = anime_list["start_date"].str.split(",", expand=True)[1]

nan_year_idx = anime_list[anime_list["start_yaer"].isna()].index
anime_list.loc[nan_year_idx, "start_yaer"] = anime_list.loc[nan_year_idx, "start_date"]
anime_list["start_yaer"] = anime_list["start_yaer"].astype(int)

### Qualitative evaluation

- Pick up random user


In [6]:
def search_anime(df, search_word):
    result = df[df["Name"].str.contains(search_word, case=False)]
    print(result[["anime_id", "Name"]].values)


def show_random_user(df, iid):
    return df[(df["anime_id"] == iid) & (df["rating"] == 10)].head(10)


def show_user_info(df, uid):
    return df[df["user_id"] == uid].sort_values("rating", ascending=False).head(10)


def predict(uid, df, anime_list):
    # predict recommend item
    result_df = model.rank(user=uid, items=anime_list["anime_id"])
    result_df = pd.DataFrame(result_df, columns=["anime_id"])
    result_df = pd.merge(result_df, anime_list, on=["anime_id"], how="inner")

    # remove rated item
    rated_df = df[df["user_id"] == uid]["anime_id"]
    result_df = result_df[~result_df["anime_id"].isin(rated_df)]
    result_df = result_df.reset_index(drop=True)[0:10]

    return result_df

#### ジャンプ系


In [7]:
search_anime(anime_list, "kimetsu")

[[38000 'Kimetsu no Yaiba']]


In [8]:
show_random_user(df, 38000)

,user_id,anime_id,rating,Name,English_name,Japanese_name,Genres,Aired,Score,Ranked,sypnopsis
458313,20,38000,10,Kimetsu no Yaiba,Demon Slayer:Kimetsu no Yaiba,鬼滅の刃,"Action, Demons, Historical, Shounen, Supernatural","Apr 6, 2019 to Sep 28, 2019",8.62,62.0,"Ever since the death of his father, the burden..."
458314,25,38000,10,Kimetsu no Yaiba,Demon Slayer:Kimetsu no Yaiba,鬼滅の刃,"Action, Demons, Historical, Shounen, Supernatural","Apr 6, 2019 to Sep 28, 2019",8.62,62.0,"Ever since the death of his father, the burden..."
458317,47,38000,10,Kimetsu no Yaiba,Demon Slayer:Kimetsu no Yaiba,鬼滅の刃,"Action, Demons, Historical, Shounen, Supernatural","Apr 6, 2019 to Sep 28, 2019",8.62,62.0,"Ever since the death of his father, the burden..."
458324,82,38000,10,Kimetsu no Yaiba,Demon Slayer:Kimetsu no Yaiba,鬼滅の刃,"Action, Demons, Historical, Shounen, Supernatural","Apr 6, 2019 to Sep 28, 2019",8.62,62.0,"Ever since the death of his father, the burden..."
458325,83,38000,10,Kimetsu no Yaiba,Demon Slayer:Kimetsu no Yaiba,鬼滅の刃,"Action, Demons, Historical, Shounen, Supernatural","Apr 6, 2019 to Sep 28, 2019",8.62,62.0,"Ever since the death of his father, the burden..."
458327,110,38000,10,Kimetsu no Yaiba,Demon Slayer:Kimetsu no Yaiba,鬼滅の刃,"Action, Demons, Historical, Shounen, Supernatural","Apr 6, 2019 to Sep 28, 2019",8.62,62.0,"Ever since the death of his father, the burden..."
458328,111,38000,10,Kimetsu no Yaiba,Demon Slayer:Kimetsu no Yaiba,鬼滅の刃,"Action, Demons, Historical, Shounen, Supernatural","Apr 6, 2019 to Sep 28, 2019",8.62,62.0,"Ever since the death of his father, the burden..."
458331,120,38000,10,Kimetsu no Yaiba,Demon Slayer:Kimetsu no Yaiba,鬼滅の刃,"Action, Demons, Historical, Shounen, Supernatural","Apr 6, 2019 to Sep 28, 2019",8.62,62.0,"Ever since the death of his father, the burden..."
458335,136,38000,10,Kimetsu no Yaiba,Demon Slayer:Kimetsu no Yaiba,鬼滅の刃,"Action, Demons, Historical, Shounen, Supernatural","Apr 6, 2019 to Sep 28, 2019",8.62,62.0,"Ever since the death of his father, the burden..."
458337,139,38000,10,Kimetsu no Yaiba,Demon Slayer:Kimetsu no Yaiba,鬼滅の刃,"Action, Demons, Historical, Shounen, Supernatural","Apr 6, 2019 to Sep 28, 2019",8.62,62.0,"Ever since the death of his father, the burden..."


In [9]:
show_user_info(df, 20)

,user_id,anime_id,rating,Name,English_name,Japanese_name,Genres,Aired,Score,Ranked,sypnopsis
31155246,20,39533,10,Given,Unknown,ギヴン,"Drama, Music, Romance, Shounen Ai, Slice of Life","Jul 12, 2019 to Sep 20, 2019",8.32,214.0,"Tightly clutching his Gibson guitar, Mafuyu Sa..."
2465104,20,35849,10,Darling in the FranXX,DARLING in the FRANXX,ダーリン・イン・ザ・フランキス,"Action, Drama, Mecha, Romance, Sci-Fi","Jan 13, 2018 to Jul 7, 2018",7.33,2210.0,"In the distant future, humanity has been drive..."
7884425,20,35790,10,Tate no Yuusha no Nariagari,The Rising of the Shield Hero,盾の勇者の成り上がり,"Action, Adventure, Drama, Fantasy","Jan 9, 2019 to Jun 26, 2019",8.00,537.0,The Four Cardinal Heroes are a group of ordina...
4571650,20,30276,10,One Punch Man,One Punch Man,ワンパンマン,"Action, Sci-Fi, Comedy, Parody, Super Power, S...","Oct 5, 2015 to Dec 21, 2015",8.57,81.0,The seemingly ordinary and unimpressive Saitam...
458313,20,38000,10,Kimetsu no Yaiba,Demon Slayer:Kimetsu no Yaiba,鬼滅の刃,"Action, Demons, Historical, Shounen, Supernatural","Apr 6, 2019 to Sep 28, 2019",8.62,62.0,"Ever since the death of his father, the burden..."
3877566,20,37430,10,Tensei shitara Slime Datta Ken,That Time I Got Reincarnated as a Slime,転生したらスライムだった件,"Fantasy, Shounen","Oct 2, 2018 to Mar 19, 2019",8.07,458.0,Thirty-seven-year-old Satoru Mikami is a typic...
2901207,20,28891,10,Haikyuu!! Second Season,Haikyu!! 2nd Season,ハイキュー!! セカンドシーズン,"Comedy, Sports, Drama, School, Shounen","Oct 4, 2015 to Mar 27, 2016",8.73,38.0,Following their participation at the Inter-Hig...
19683615,20,552,10,Digimon Adventure,Digimon:Digital Monsters,デジモンアドベンチャー,"Action, Adventure, Comedy, Fantasy, Kids","Mar 7, 1999 to Mar 26, 2000",7.79,839.0,hen a group of seven children go to summer cam...
24611509,20,32615,10,Youjo Senki,The Saga of Tanya the Evil,幼女戦記,"Action, Military, Magic","Jan 6, 2017 to Mar 31, 2017",8.00,547.0,Tanya Degurechaff is a young soldier infamous ...
1475988,20,32935,10,Haikyuu!!: Karasuno Koukou vs. Shiratorizawa G...,Haikyu!! 3rd Season,ハイキュー!! 烏野高校 VS 白鳥沢学園高校,"Comedy, Sports, Drama, School, Shounen","Oct 8, 2016 to Dec 10, 2016",8.87,20.0,"fter the victory against Aoba Jousai High, Kar..."


In [10]:
predict(20, df, anime_list[anime_list["start_yaer"] >= 2010])
# predict(20, df, anime_list)

,anime_id,Name,Japanese_name,Aired,Score,start_date,end_date,start_yaer
0,31240,Re:Zero kara Hajimeru Isekai Seikatsu,Re：ゼロから始める異世界生活,"Apr 4, 2016 to Sep 19, 2016",8.29,"Apr 4, 2016","Sep 19, 2016",2016
1,32182,Mob Psycho 100,モブサイコ100,"Jul 11, 2016 to Sep 27, 2016",8.49,"Jul 11, 2016","Sep 27, 2016",2016
2,37450,Seishun Buta Yarou wa Bunny Girl Senpai no Yum...,青春ブタ野郎はバニーガール先輩の夢を見ない,"Oct 4, 2018 to Dec 27, 2018",8.38,"Oct 4, 2018","Dec 27, 2018",2018
3,23755,Nanatsu no Taizai,七つの大罪,"Oct 5, 2014 to Mar 29, 2015",7.89,"Oct 5, 2014","Mar 29, 2015",2014
4,13759,Sakura-sou no Pet na Kanojo,さくら荘のペットな彼女,"Oct 9, 2012 to Mar 26, 2013",8.19,"Oct 9, 2012","Mar 26, 2013",2012
5,9253,Steins;Gate,STEINS;GATE,"Apr 6, 2011 to Sep 14, 2011",9.11,"Apr 6, 2011","Sep 14, 2011",2011
6,39587,Re:Zero kara Hajimeru Isekai Seikatsu 2nd Season,Re：ゼロから始める異世界生活,"Jul 8, 2020 to Sep 30, 2020",8.50,"Jul 8, 2020","Sep 30, 2020",2020
7,10620,Mirai Nikki,未来日記,"Oct 9, 2011 to Apr 15, 2012",7.54,"Oct 9, 2011","Apr 15, 2012",2011
8,22199,Akame ga Kill!,アカメが斬る！,"Jul 7, 2014 to Dec 15, 2014",7.53,"Jul 7, 2014","Dec 15, 2014",2014
9,30503,Noragami Aragoto,ノラガミ ARAGOTO,"Oct 3, 2015 to Dec 26, 2015",8.22,"Oct 3, 2015","Dec 26, 2015",2015


#### 女性向け


In [11]:
search_anime(anime_list, "inu")

[[34542 'Inuyashiki']
 [249 'InuYasha']
 [17831 'Inu to Hasami wa Tsukaiyou']
 [22123 'Inugami-san to Nekoyama-san']
 [11013 'Inu x Boku SS']
 [39523
  'Choujin Koukousei-tachi wa Isekai demo Yoyuu de Ikinuku you desu!']
 [940 'Inukami!']
 [6811 'InuYasha: Kanketsu-hen']
 [8449 'Togainu no Chi']
 [13141 'Shiba Inuko-san']
 [31387 'Aru Hi Inu no Kuni kara Tegami ga Kite']
 [37890 'Oshi ga Budoukan Ittekuretara Shinu']
 [2553 'Flanders no Inu']
 [8920 'Inuki Kanako Zekkyou Collection: Gakkou ga Kowai!']
 [2749 'Heisei Inu Monogatari Bow']
 [2624 'Flanders no Inu, Boku no Patrasche']
 [1839 'Jagainu-kun']]


In [12]:
show_random_user(df, 11013)

,user_id,anime_id,rating,Name,English_name,Japanese_name,Genres,Aired,Score,Ranked,sypnopsis
28533638,61,11013,10,Inu x Boku SS,Inu X Boku Secret Service,妖狐×僕SS,"Comedy, Supernatural, Romance, Shounen","Jan 13, 2012 to Mar 30, 2012",7.46,1693.0,chiyo Shirakiin is the sheltered daughter of a...
28533643,90,11013,10,Inu x Boku SS,Inu X Boku Secret Service,妖狐×僕SS,"Comedy, Supernatural, Romance, Shounen","Jan 13, 2012 to Mar 30, 2012",7.46,1693.0,chiyo Shirakiin is the sheltered daughter of a...
28533645,110,11013,10,Inu x Boku SS,Inu X Boku Secret Service,妖狐×僕SS,"Comedy, Supernatural, Romance, Shounen","Jan 13, 2012 to Mar 30, 2012",7.46,1693.0,chiyo Shirakiin is the sheltered daughter of a...
28533657,267,11013,10,Inu x Boku SS,Inu X Boku Secret Service,妖狐×僕SS,"Comedy, Supernatural, Romance, Shounen","Jan 13, 2012 to Mar 30, 2012",7.46,1693.0,chiyo Shirakiin is the sheltered daughter of a...
28533658,281,11013,10,Inu x Boku SS,Inu X Boku Secret Service,妖狐×僕SS,"Comedy, Supernatural, Romance, Shounen","Jan 13, 2012 to Mar 30, 2012",7.46,1693.0,chiyo Shirakiin is the sheltered daughter of a...
28533661,286,11013,10,Inu x Boku SS,Inu X Boku Secret Service,妖狐×僕SS,"Comedy, Supernatural, Romance, Shounen","Jan 13, 2012 to Mar 30, 2012",7.46,1693.0,chiyo Shirakiin is the sheltered daughter of a...
28533663,310,11013,10,Inu x Boku SS,Inu X Boku Secret Service,妖狐×僕SS,"Comedy, Supernatural, Romance, Shounen","Jan 13, 2012 to Mar 30, 2012",7.46,1693.0,chiyo Shirakiin is the sheltered daughter of a...
28533664,320,11013,10,Inu x Boku SS,Inu X Boku Secret Service,妖狐×僕SS,"Comedy, Supernatural, Romance, Shounen","Jan 13, 2012 to Mar 30, 2012",7.46,1693.0,chiyo Shirakiin is the sheltered daughter of a...
28533670,351,11013,10,Inu x Boku SS,Inu X Boku Secret Service,妖狐×僕SS,"Comedy, Supernatural, Romance, Shounen","Jan 13, 2012 to Mar 30, 2012",7.46,1693.0,chiyo Shirakiin is the sheltered daughter of a...
28533682,457,11013,10,Inu x Boku SS,Inu X Boku Secret Service,妖狐×僕SS,"Comedy, Supernatural, Romance, Shounen","Jan 13, 2012 to Mar 30, 2012",7.46,1693.0,chiyo Shirakiin is the sheltered daughter of a...


In [13]:
show_user_info(df, 457)

,user_id,anime_id,rating,Name,English_name,Japanese_name,Genres,Aired,Score,Ranked,sypnopsis
263210,457,2034,10,Lovely★Complex,Lovely Complex,ラブ★コン,"Comedy, Romance, Shoujo","Apr 7, 2007 to Sep 29, 2007",8.06,472.0,ove is unusual for Koizumi Risa and Ootani Ats...
28533682,457,11013,10,Inu x Boku SS,Inu X Boku Secret Service,妖狐×僕SS,"Comedy, Supernatural, Romance, Shounen","Jan 13, 2012 to Mar 30, 2012",7.46,1693.0,chiyo Shirakiin is the sheltered daughter of a...
30162955,457,18661,10,Kamisama Hajimemashita OVA,Unknown,神様はじめました,"Comedy, Demons, Supernatural, Romance, Fantasy...","Aug 20, 2013",7.98,560.0,Two OVA episodes bundled with the 16th volume ...
17777817,457,1562,10,Yamato Nadeshiko Shichihenge♥,The Seven Metamorphoses of Yamato Nadeshiko,ヤマトナデシコ七変化♥,"Comedy, Shoujo","Oct 4, 2006 to Mar 28, 2007",7.76,902.0,Ever since her crush rejected her by insulting...
6058331,457,23755,10,Nanatsu no Taizai,The Seven Deadly Sins,七つの大罪,"Action, Adventure, Ecchi, Fantasy, Magic, Shou...","Oct 5, 2014 to Mar 29, 2015",7.89,690.0,In a world similar to the European Middle Ages...
17643152,457,120,10,Fruits Basket,Fruits Basket,フルーツバスケット,"Slice of Life, Comedy, Supernatural, Drama, Ro...","Jul 5, 2001 to Dec 27, 2001",7.69,1058.0,fter the accident in which she lost her mother...
6314796,457,22319,10,Tokyo Ghoul,Tokyo Ghoul,東京喰種-トーキョーグール-,"Action, Mystery, Horror, Psychological, Supern...","Jul 4, 2014 to Sep 19, 2014",7.81,807.0,Tokyo has become a cruel and merciless city—a ...
29993289,457,25781,10,Shingeki no Kyojin: Kuinaki Sentaku,Unknown,進撃の巨人 悔いなき選択,"Action, Fantasy, Shoujo","Dec 9, 2014 to Apr 9, 2015",8.38,170.0,any years before becoming the famed captain of...
33961906,457,14645,10,Hiiro no Kakera Dai Ni Shou,Hiiro no Kakera:The Tamayori Princess Saga Sea...,緋色の欠片 第二章,"Fantasy, Romance, Shoujo, Supernatural","Sep 30, 2012 to Dec 23, 2012",7.23,2651.0,Second season of Hiiro no Kakera.
8469589,457,14227,10,Tonari no Kaibutsu-kun,My Little Monster,となりの怪物くん,"Slice of Life, Comedy, Romance, School, Shoujo","Oct 2, 2012 to Dec 25, 2012",7.55,1417.0,Shizuku Mizutani is apathetic towards her clas...


In [14]:
predict(457, df, anime_list[anime_list["start_yaer"] >= 2010])
# predict(457, df, anime_list)

,anime_id,Name,Japanese_name,Aired,Score,start_date,end_date,start_yaer
0,37962,IDOLiSH7: Second Beat!,アイドリッシュセブン Second BEAT!,"Apr 5, 2020 to Dec 27, 2020",8.15,"Apr 5, 2020","Dec 27, 2020",2020
1,34787,Idol Time PriPara,アイドルタイムプリパラ,"Apr 4, 2017 to Mar 27, 2018",7.13,"Apr 4, 2017","Mar 27, 2018",2017
2,39699,King of Prism: Shiny Seven Stars,KING OF PRISM -Shiny Seven Stars-,"Apr 16, 2019 to Jul 2, 2019",6.40,"Apr 16, 2019","Jul 2, 2019",2019
3,40901,Toji no Miko: Kizamishi Issen no Tomoshibi OVA,刀使ノ巫女 刻みし一閃の燈火,"Oct 25, 2020 to Nov 29, 2020",6.55,"Oct 25, 2020","Nov 29, 2020",2020
4,36967,Lupin III: Lupin wa Ima mo Moete Iru ka?,ルパンは今も燃えているか？,"Jul 25, 2018",6.97,"Jul 25, 2018",None,2018
5,17137,Saiyuuki Gaiden: Kouga no Shou,最遊記外伝 特別編 香花の章,"Apr 26, 2013",7.28,"Apr 26, 2013",None,2013
6,8287,SD Gundam Sangokuden Brave Battle Warriors,SDガンダム三国伝 Brave Battle Warriors,"Apr 3, 2010 to Mar 19, 2011",6.34,"Apr 3, 2010","Mar 19, 2011",2010
7,32717,Aikatsu Stars!,アイカツスターズ！,"Apr 7, 2016 to Mar 29, 2018",7.52,"Apr 7, 2016","Mar 29, 2018",2016
8,25801,Shouwa Genroku Rakugo Shinjuu: Yotarou Hourou-hen,昭和元禄落語心中 与太郎放浪篇,"Mar 6, 2015 to Aug 7, 2015",7.60,"Mar 6, 2015","Aug 7, 2015",2015
9,9127,Zettai Karen Children: Gentei Kaikin!! OVA Cho...,絶対可憐チルドレン 限定解禁!! OVA超先取りスペシャル!!,"Apr 16, 2010",6.80,"Apr 16, 2010",None,2010
